In [11]:
import os, shutil
import tensorflow as tf
import numpy as np
from keras import models
from keras.models import load_model
from keras.preprocessing import image

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

IMAGE_HEIGHT = 60
IMAGE_WIDTH = 180
test_num = 10000
MAXLABEL = 40
base_dir = 'D:\src\captchagen\out'


def convert2gray(img):
    if len(img.shape) > 2:
        gray = np.mean(img, -1)
        return gray
    else:
        return img
    
def num2vec(num):
    if num < 0 or num > 9999 :
        raise ValueError('number not in range 0-9999')
    vector = np.zeros(MAXLABEL)
    i = 0
    while int(num) > 0:
        idx = i * 10 + int(num) % 10
        #print(idx)
        num /= 10
        i += 1
        vector[idx] = 1 
    return vector

test_dir = os.path.join(base_dir, 'test')
test_set = np.ndarray(shape=(test_num, IMAGE_HEIGHT, IMAGE_WIDTH),dtype=np.float32)
test_label = np.ndarray(shape=(test_num, MAXLABEL),dtype=np.float32)
test_num = 10000
for i in range(test_num) :
    fname = '{}.png'.format(i)
    file = os.path.join(test_dir,fname)
    img = image.load_img(file, target_size=(IMAGE_HEIGHT,IMAGE_WIDTH))
    arr = np.asarray(img,dtype="float32")/255.0
    arr = convert2gray(arr)
    test_set[i] = arr    
    test_label[i] = num2vec(i)

test_set = test_set.reshape((test_num,IMAGE_HEIGHT, IMAGE_WIDTH,1))

In [31]:
model = load_model('weights-improvement-226-0.97.hdf5')
real_label = model.predict(test_set)

array([2, 1, 8, 4], dtype=int64)

In [32]:
alabel = real_label[812].reshape(4,10)
numvec = alabel.argmax(axis=1)
numvec

array([2, 1, 8, 4], dtype=int64)

In [35]:
acc = 0
for i in range(10000):
    alabel = real_label[i].reshape(4,10)
    numvec = alabel.argmax(axis=1)
    num = numvec[3]*1000 +  numvec[2]*100 + numvec[1]*10 + numvec[0]
    if num == i:
        acc +=1
        
acc

8713